In [2]:
import requests
import pandas as pd
from io import StringIO

url = "https://portaldades.ajuntament.barcelona.cat/services/backend/rest/statistic/export?id=ydbku9mpnx&fileformat=CSV"
headers = {"X-IBM-Client-Id": "f943a426a663183091a1401f55eec60d"}

response = requests.get(url, headers=headers)
print("✅ API'den veri çekiliyor:")

if response.status_code == 200:
    # Veriyi StringIO ile oku
    data = StringIO(response.content.decode('latin1'))
    df_tenure_regime = pd.read_csv(data, sep=',', encoding='latin1')  # SEP = ','
else:
    print(f"Veri çekilemedi! Status code: {response.status_code}")


print("✅ API'den veri çekildi:")
print(df_tenure_regime.head())


✅ API'den veri çekiliyor:
✅ API'den veri çekildi:
           Dim-00:TEMPS Dim-01:TERRITORI  Dim-01:TERRITORI (order)  \
0  1981-01-01T00:00:00Z        Barcelona                         1   
1  1981-01-01T00:00:00Z        Barcelona                         1   
2  1981-01-01T00:00:00Z        Barcelona                         1   
3  1991-01-01T00:00:00Z        Barcelona                         1   
4  1991-01-01T00:00:00Z  Horta-GuinardÃ³                         7   

  Dim-01:TERRITORI (type) Dim-02:RÃGIM DE TINENÃA     VALUE  
0                Municipi                   Lloguer  268881.0  
1                Municipi                 Propietat  287153.0  
2                Municipi                    Altres   23797.0  
3                Municipi                 Propietat  354477.0  
4               Districte                   Lloguer   17872.0  


In [3]:
df_tenure_regime.columns

Index(['Dim-00:TEMPS', 'Dim-01:TERRITORI', 'Dim-01:TERRITORI (order)',
       'Dim-01:TERRITORI (type)', 'Dim-02:RÃGIM DE TINENÃA', 'VALUE'],
      dtype='object')

In [4]:
df_tenure_regime.rename(columns={
    'Dim-00:TEMPS': 'year',
    'Dim-01:TERRITORI': 'territory',
    'Dim-01:TERRITORI (order)': 'territory_order',
    'Dim-01:TERRITORI (type)': 'territory_type',
    'Dim-02:RÃGIM DE TINENÃA': 'tenure_regime',
    'VALUE': 'num_main_homes'
}, inplace=True)


In [5]:
df_tenure_regime.head()

,year,territory,territory_order,territory_type,tenure_regime,num_main_homes
0,1981-01-01T00:00:00Z,Barcelona,1,Municipi,Lloguer,268881.0
1,1981-01-01T00:00:00Z,Barcelona,1,Municipi,Propietat,287153.0
2,1981-01-01T00:00:00Z,Barcelona,1,Municipi,Altres,23797.0
3,1991-01-01T00:00:00Z,Barcelona,1,Municipi,Propietat,354477.0
4,1991-01-01T00:00:00Z,Horta-GuinardÃ³,7,Districte,Lloguer,17872.0


In [6]:
import numpy as np
for column in df_tenure_regime:
    unique_vals = np.unique(df_tenure_regime[column].fillna('0'))
    nr_values = len(unique_vals)
    if nr_values <= 85:
        print("The number of values for feature {} :{} -- {}".format(column, nr_values, unique_vals))
    else:
        print("The number of values for feature {} :{}".format(column, nr_values))

The number of values for feature year :5 -- ['1981-01-01T00:00:00Z' '1991-01-01T00:00:00Z' '2001-01-01T00:00:00Z'
 '2011-01-01T00:00:00Z' '2021-01-01T00:00:00Z']
The number of values for feature territory :83 -- ['Barcelona' 'BarÃ³ de Viver' 'Can BarÃ³' 'Can Peguera' 'Canyelles'
 'Ciutat Meridiana' 'Ciutat Vella'
 'Diagonal Mar i el Front MarÃ\xadtim del Poblenou' 'Eixample'
 'GrÃ\xa0cia' 'Horta' 'Horta-GuinardÃ³' 'Hostafrancs' 'Les Corts'
 'Montbau' 'Navas' 'Nou Barris' 'Pedralbes' 'Porta'
 'ProvenÃ§als del Poblenou' 'Sant Andreu' 'Sant Antoni'
 'Sant GenÃ\xads dels Agudells' 'Sant Gervasi - Galvany'
 'Sant Gervasi - la Bonanova' 'Sant MartÃ\xad'
 'Sant MartÃ\xad de ProvenÃ§als' 'Sant Pere, Santa Caterina i la Ribera'
 'Sants' 'Sants - Badal' 'Sants-MontjuÃ¯c' 'SarriÃ\xa0'
 'SarriÃ\xa0-Sant Gervasi' 'Torre BarÃ³' 'Vallbona'
 'Vallcarca i els Penitents' 'Vallvidrera, el Tibidabo i les Planes'
 'Verdun' 'Vilapicina i la Torre Llobeta' 'el Baix GuinardÃ³'
 'el Barri GÃ²tic' 'el BesÃ²s i 

In [ ]:
df_tenure_regime['territory'] = df_tenure_regime['territory'].str.encode('latin1').str.decode('utf-8')
df_tenure_regime['territory_type'] = df_tenure_regime['territory_type'].str.encode('latin1').str.decode('utf-8')

In [8]:
print(df_tenure_regime['territory'].unique()[:10])
print(df_tenure_regime['territory_type'].unique())


['Barcelona' 'Horta-Guinardó' 'Ciutat Vella' 'Nou Barris' 'Sants-Montjuïc'
 'Sant Martí' 'Gràcia' 'Eixample' 'Sant Andreu' 'Les Corts']
['Municipi' 'Districte' 'Barri']


In [9]:
print(sorted(df_tenure_regime['year'].unique()))

['1981-01-01T00:00:00Z', '1991-01-01T00:00:00Z', '2001-01-01T00:00:00Z', '2011-01-01T00:00:00Z', '2021-01-01T00:00:00Z']


In [10]:
df_tenure_regime['year'] = pd.to_datetime(df_tenure_regime['year'], errors='coerce').dt.year

In [11]:
print(sorted(df_tenure_regime['year'].unique()))

[1981, 1991, 2001, 2011, 2021]


In [12]:
df_tenure_regime.head()

,year,territory,territory_order,territory_type,tenure_regime,num_main_homes
0,1981,Barcelona,1,Municipi,Lloguer,268881.0
1,1981,Barcelona,1,Municipi,Propietat,287153.0
2,1981,Barcelona,1,Municipi,Altres,23797.0
3,1991,Barcelona,1,Municipi,Propietat,354477.0
4,1991,Horta-Guinardó,7,Districte,Lloguer,17872.0


In [13]:
df_tenure_regime['tenure_regime'].unique()

array(['Lloguer', 'Propietat', 'Altres'], dtype=object)

In [14]:
df_tenure_regime['tenure_regime'] = df_tenure_regime['tenure_regime'].replace({
    'Lloguer': 'Rent',
    'Propietat': 'Ownership',
    'Altres': 'Other'
})


In [15]:
print(df_tenure_regime['tenure_regime'].unique())

['Rent' 'Ownership' 'Other']


In [16]:
df_tenure_regime.head()

,year,territory,territory_order,territory_type,tenure_regime,num_main_homes
0,1981,Barcelona,1,Municipi,Rent,268881.0
1,1981,Barcelona,1,Municipi,Ownership,287153.0
2,1981,Barcelona,1,Municipi,Other,23797.0
3,1991,Barcelona,1,Municipi,Ownership,354477.0
4,1991,Horta-Guinardó,7,Districte,Rent,17872.0


In [17]:
df_tenure_regime.to_csv("tenure_regime_count.csv", index=False, encoding='utf-8')
